In [ ]:
!unzip -q /content/drive/MyDrive/Places2_simp.zip -d /content/drive/MyDrive/

In [ ]:
from PIL import Image
image = Image.open('/content/drive/MyDrive/Places2_simp/airport_terminal/00000001.jpg')
image.show()


In [ ]:
from PIL import Image
import os
import numpy as np
import random
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

# Define the target size for the resized images
target_size = (256, 256)

# Define the transforms for data augmentation
transform_train = transforms.Compose([
    transforms.RandomRotation(degrees=20),
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.Resize(target_size),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Set the path to the dataset directory
dataset_dir = '/content/drive/MyDrive/Places2_simp'

# Get a list of all image file paths in the dataset directory
file_paths = []
class_names = sorted(os.listdir(dataset_dir))
for class_name in class_names:
    class_dir = os.path.join(dataset_dir, class_name)
    for file_name in os.listdir(class_dir):
        file_path = os.path.join(class_dir, file_name)
        file_paths.append((file_path, class_names.index(class_name)))

# Split the dataset into a training and validation set
train_file_paths, val_file_paths = train_test_split(file_paths, test_size=0.2, random_state=42)

# Define a function to load an image and apply the data augmentation transforms
def load_image(file_path, transform):
    # Open the image file
    image = Image.open(file_path)

    # Apply the data augmentation transforms
    if transform:
        image = transform(image)

    return image

# Define a function to create a data loader object for a given set of file paths
def create_data_loader(file_paths, transform, batch_size, shuffle):
    return DataLoader(
        [(file_path, label) for file_path, label in file_paths],
        batch_size=batch_size,
        shuffle=shuffle,
        num_workers=2,
        collate_fn=lambda batch: (
            torch.stack([load_image(file_path, transform) for file_path, _ in batch]),
            torch.tensor([label for _, label in batch])
        )
    )

# Create data loader objects for the training and validation sets
train_loader = create_data_loader(train_file_paths, transform_train, batch_size=16, shuffle=True)
val_loader = create_data_loader(val_file_paths, transform_test, batch_size=16, shuffle=False)


In [ ]:
import os

dir_path = '/content/drive/MyDrive/Places2_simp'
if not os.path.exists(dir_path):
    print(f"The directory {dir_path} does not exist.")
else:
    print(f"The directory {dir_path} exists.")


In [ ]:
print(f"Number of images in the training set: {len(train_file_paths)}")
print(f"Number of images in the validation set: {len(val_file_paths)}")

In [ ]:
from torchvision.models.resnet import resnet34
import torch
import torch.nn as nn
import torchvision.models as models
import torch.optim.lr_scheduler as lr_scheduler

# Load the ResNet-34 model and remove the last layers
resnet34 = models.resnet34(pretrained=True)
modules = list(resnet34.children())[:-2]
backbone = nn.Sequential(*modules)

# Define the new layers for classification
num_classes = 40
fc_layers = nn.Sequential(
    nn.AdaptiveAvgPool2d((1, 1)),
    nn.Flatten(),
    nn.Linear(512, 256),
    nn.ReLU(inplace=True),
    nn.Linear(256, num_classes)
)

# Combine the ResNet-34 backbone with the new layers
model = nn.Sequential(
    backbone,
    fc_layers
)


# Set a learning rate
optimizer = torch.optim.Adam([
    {'params': backbone.parameters(), 'lr': 0.0001},
    {'params': fc_layers.parameters(), 'lr': 0.0001},
])
scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

# Define the loss function
criterion = nn.CrossEntropyLoss()


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torch.optim.lr_scheduler as lr_scheduler

# Check if a GPU is available and set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

resnet = models.resnet34(weights=models.resnet.ResNet34_Weights.IMAGENET1K_V1)

for param in resnet.parameters():
    param.requires_grad = False

num_classes = 40

# Get the number of input features for the last layer
num_features = resnet.fc.in_features

# Replace the last layer with a new fully connected layer
resnet.fc = nn.Linear(num_features, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW([
    {'params': resnet.fc.parameters(), 'lr': 0.01},  # Learning rate for new layers
    {'params': resnet.layer4.parameters(), 'lr': 0.001},  # Learning rate for transferred layers
    {'params': resnet.layer3.parameters(), 'lr': 0.001},  # Learning rate for transferred layers
    {'params': resnet.layer2.parameters(), 'lr': 0.001},  # Learning rate for transferred layers
    {'params': resnet.layer1.parameters(), 'lr': 0.001},  # Learning rate for transferred layers
    {'params': resnet.conv1.parameters(), 'lr': 0.001},  # Learning rate for transferred layers
], lr=0.001, betas=(0.9, 0.999), eps=1e-8, amsgrad=True)

scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

# Define the loss function
criterion = nn.CrossEntropyLoss()

# Move the model to the device
resnet = resnet.to(device)

In [ ]:
!pip install tensorflow
!pip install tensorboard


In [ ]:

from torch.utils.tensorboard import SummaryWriter

# Create a SummaryWriter object
writer = SummaryWriter('logs')

# Define the number of epochs and the batch size
num_epochs = 10
batch_size = 4

# Loop over the data and train the model
for epoch in range(num_epochs):
    print(f'Epoch {epoch+1}/{num_epochs}')
    print('-' * 40)
    train_loss = 0.0
    train_correct = 0
    train_total = 0
    resnet.train() # set the model to training model
    for batch_idx, (data, target) in enumerate(train_loader):

        # Move the data and target tensors to the device
        data = data.to(device)
        target = target.to(device)

        # Forward pass
        output = resnet(data)
        loss = criterion(output, target)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Log the loss
        writer.add_scalar('Loss/train', loss.item(), epoch * len(train_loader) + batch_idx)
        train_loss += loss.item()

        # Calculate training accuracy
        _, predicted = torch.max(output.data, 1)
        train_total += target.size(0)
        train_correct += (predicted == target).sum().item()

        # Calculate batch accuracy and print training statistics
        batch_accuracy = 100 * (predicted == target).sum().item() / target.size(0)
        if batch_idx % 10 == 0:
            print(f'Train Batch {batch_idx}/{len(train_loader)} - loss: {loss.item():.4f} - batch accuracy: {batch_accuracy:.2f}%')

    # Log the average training loss and accuracy
    train_loss /= len(train_loader)
    train_accuracy = 100 * train_correct / train_total
    writer.add_scalar('Loss/train_avg', train_loss, epoch)
    writer.add_scalar('Accuracy/train', train_accuracy, epoch)
    print(f'Train loss: {train_loss:.4f} - Train accuracy: {train_accuracy:.2f}%')

    # Evaluate on validation set
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    resnet.eval() # set the model to evaluation mode
    with torch.no_grad():
        for data, target in val_loader:
            # Move the data and target tensors to the device
            data = data.to(device)
            target = target.to(device)

            output = resnet(data)
            loss = criterion(output, target)
            val_loss += loss.item()
            _, predicted = torch.max(output.data, 1)
            val_total += target.size(0)
            val_correct += (predicted == target).sum().item()

    # Log the average validation loss and accuracy
    val_loss /= len(val_loader)
    val_accuracy = 100 * val_correct / val_total
    writer.add_scalar('Loss/val', val_loss, epoch)
    writer.add_scalar('Accuracy/val', val_accuracy, epoch)
    print(f'Validation loss: {val_loss:.4f} - Validation accuracy: {val_accuracy:.2f}%')
    print('=' * 40)

    # Update the learning rate
    scheduler.step()

# Close the SummaryWriter
writer.close()

In [ ]:
import torch
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter


# Create a SummaryWriter object
writer = SummaryWriter('logs')

# Define the number of epochs and the batch size
num_epochs = 6
batch_size = 16


# Loop over the data and train the model
for epoch in range(num_epochs):
    print(f'Epoch {epoch+1}/{num_epochs}')
    print('-' * 40)
    train_loss = 0.0
    train_correct = 0
    train_total = 0
    model.train() # set the model to training mode
    for batch_idx, (data, target) in enumerate(train_loader):
        try:
            # Move the data and target tensors to the device
            data = data.to(device)
            target = target.to(device)

            # Forward pass
            output = model(data)
            loss = criterion(output, target)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Log the loss
            writer.add_scalar('Loss/train', loss.item(), epoch * len(train_loader) + batch_idx)
            train_loss += loss.item()

            # Calculate training accuracy
            _, predicted = torch.max(output.data, 1)
            train_total += target.size(0)
            train_correct += (predicted == target).sum().item()

            # Calculate batch accuracy and print training statistics
            batch_accuracy = 100 * (predicted == target).sum().item() / target.size(0)
            if batch_idx % 10 == 0:
                print(f'Train Batch {batch_idx}/{len(train_loader)} - loss: {loss.item():.4f} - batch accuracy: {batch_accuracy:.2f}%')

        except FileNotFoundError:
            print(f'File not found. Skipping batch {batch_idx}/{len(train_loader)}.')

    # Log the average training loss and accuracy
    train_loss /= len(train_loader)
    train_accuracy = 100 * train_correct / train_total
    writer.add_scalar('Loss/train_avg', train_loss, epoch)
    writer.add_scalar('Accuracy/train', train_accuracy, epoch)
    print(f'Train loss: {train_loss:.4f} - Train accuracy: {train_accuracy:.2f}%')

    # Evaluate on the validation set
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    model.eval() # set the model to evaluation mode
    with torch.no_grad():
        for data, target in val_loader:
            # Move the data and target tensors to the device
            data = data.to(device)
            target = target.to(device)

            output = model(data)
            loss = criterion(output, target)
            val_loss += loss.item()
            _, predicted = torch.max(output.data, 1)
            val_total += target.size(0)
            val_correct += (predicted == target).sum().item()

    # Log the average validation loss and accuracy
    val_loss /= len(val_loader)
    val_accuracy = 100 * val_correct / val_total
    writer.add_scalar('Loss/val', val_loss, epoch)
    writer.add_scalar('Accuracy/val', val_accuracy, epoch)
    print(f'Validation loss: {val_loss:.4f} - Validation accuracy: {val_accuracy:.2f}%')
    print('=' * 40)

    # Update the learning rate
    scheduler.step()

# Close the SummaryWriter
writer.close()


In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix
top1_correct = 0
top5_correct = 0
total = 0
predicted_labels = []
target_labels = []
model.eval()
with torch.no_grad():
    for data, target in val_loader:
        output = model(data)
        _, predicted = torch.max(output.data, 1)
        _, top5_predicted = torch.topk(output, 5, dim=1)

        total += target.size(0)
        top1_correct += (predicted == target).sum().item()
        top5_correct += sum([(target[i] in top5_predicted[i]) for i in range(len(target))])

        predicted_labels.extend(predicted.tolist())
        target_labels.extend(target.tolist())
top1_accuracy = 100 * top1_correct / total
top5_accuracy = 100 * top5_correct / total
confusion = confusion_matrix(target_labels, predicted_labels)

import matplotlib.pyplot as plt
import seaborn as sns

class_names = ["airport_terminal" , "amphithetre" , "amusement_park", "art_gallery", "bakery_shop", "bar", "bookstore", "botanical_garden",
               "bridge", "bus_interior", "butshers_shop", "campsite", "classroom", "coffee_shop", "construction_site", "courtyard", "driveway",
               "fire_station", "fountain", "gas_station", "harbour", "highway", "kindergarden_classroom", "lobby", "market_outdoor", "museum",
               "office", "parking_lot", "ailroad_track", "restaurant", "river", "shed", "staircase", "supermarket", "swimming_pool_outdoor",
               "track", "valley","yard"]

plt.figure(figsize=(10, 8))
sns.heatmap(confusion, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()
num_samples = 5  # Number of samples to display
random_indices = np.random.choice(range(len(target_labels)), size=num_samples, replace=False)

for index in random_indices:
    image = val_dataset[index][0]  # Get the image tensor
    actual_class = class_names[target_labels[index]]  # Get the actual class name
    predicted_classes = [class_names[pred] for pred in top5_predicted[index]]  # Get the predicted class names

    # Display the image and class names
    plt.imshow(image.permute(1, 2, 0))
    plt.axis('off')
    plt.title(f'Top-5 Predicted: {predicted_classes}\nActual: {actual_class}')
    plt.show()
